In [1]:
iam(x::Integer) = "an integer"
iam(x::String) = "a string"

iam (generic function with 2 methods)

In [2]:
iam(1)

"an integer"

In [3]:
iam("1")

"a string"

In [4]:
iam(1.5)

MethodError: MethodError: no method matching iam(::Float64)
Closest candidates are:
  iam(!Matched::String) at In[1]:2
  iam(!Matched::Integer) at In[1]:1

In [13]:
abstract type Number ; end
abstract type Real     <: Number ; end
abstract type FloatingPoint <: Real ; end
abstract type Signd    <: Integer ; end
abstract type Unsigned <: Integer ; end

In [14]:
# composite types
struct Pixel
    x::Int64
    y::Int64
    color::Int64
end

In [15]:
p = Pixel(5, 5, 100)

Pixel(5, 5, 100)

In [16]:
p.x = 100

ErrorException: setfield! immutable struct of type Pixel cannot be changed

In [17]:
p.x

5

In [18]:
mutable struct MPixel
    x::Int64
    y::Int64
    color::Int64
end

In [19]:
p = MPixel(5, 5, 100)

MPixel(5, 5, 100)

In [20]:
p.x = 100

100

In [21]:
p.x

100

In [22]:
@show p

p = MPixel(100, 5, 100)


MPixel(100, 5, 100)

In [29]:
# 設定型別參數
struct Pixel2{T}
    x::Int64
    y::Int64
    color::T
end

In [32]:
p = Pixel2(5, 5, "yellow")

Pixel2{String}(5, 5, "yellow")

In [1]:
# 型別穩定性, 是指函數回傳值的型別應該被arguement的型別唯一決定, 而不是看回傳值本身
# 以下是一個非型別穩定的函數
function  pos(x)
    if x < 0
        return 0
    else
        return x
    end
end

pos (generic function with 1 method)

In [2]:
pos(-1),
pos(-2.5),
pos(2.5)

(0, 0, 2.5)

In [3]:
typeof(pos(2.5))

Float64

In [4]:
typeof(pos(-2.5))

Int64

In [12]:
# 一個解法是為每一個型別都定義一個函數做多重分派
# 另一個較聰明的方法是
function pos_fixed(x)
    if x < 0
        return zero(x) # 這裡回傳的0型別會跟x一樣
    else
        return x
    end
end

pos_fixed (generic function with 1 method)

In [14]:
typeof(pos_fixed(2.5))

Float64

In [15]:
typeof(pos_fixed(-2.5))

Float64

In [19]:
# 從紅色警示部分可以看出這支程式型別不穩定
@code_warntype pos(2.5)

Variables
  #self#::Core.Compiler.Const(pos, false)
  x::Float64

Body::Union{Float64, Int64}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x


In [20]:
@code_warntype pos_fixed(2.5)

Variables
  #self#::Core.Compiler.Const(pos_fixed, false)
  x::Float64

Body::Float64
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.zero(x)::Core.Compiler.Const(0.0, false)
└──      return %3
3 ─      return x


In [21]:
# 從LLVM bitcode的長度也可看出具有穩定性的函數, 內部要執行的代碼更少
@code_llvm pos(2.5)


;  @ In[1]:4 within `pos'
; Function Attrs: uwtable
define { %jl_value_t addrspace(10)*, i8 } @julia_pos_19980([8 x i8]* noalias nocapture align 8 dereferenceable(8), double) #0 {
top:
; ┌ @ float.jl:503 within `<' @ float.jl:458
   %2 = fcmp uge double %1, 0.000000e+00
; └
  br i1 %2, label %L12, label %L11

L11:                                              ; preds = %L12, %top
  %merge = phi { %jl_value_t addrspace(10)*, i8 } [ { %jl_value_t addrspace(10)* addrspacecast (%jl_value_t* inttoptr (i64 362709024 to %jl_value_t*) to %jl_value_t addrspace(10)*), i8 -126 }, %top ], [ { %jl_value_t addrspace(10)* addrspacecast (%jl_value_t* null to %jl_value_t addrspace(10)*), i8 1 }, %L12 ]
;  @ In[1]:5 within `pos'
  ret { %jl_value_t addrspace(10)*, i8 } %merge

L12:                                              ; preds = %top
;  @ In[1]:7 within `pos'
  %.0..sroa_cast = bitcast [8 x i8]* %0 to double*
  store double %1, double* %.0..sroa_cast, align 8
  br label %L11
}


In [22]:
@code_llvm pos_fixed(2.5)


;  @ In[12]:4 within `pos_fixed'
; Function Attrs: uwtable
define double @julia_pos_fixed_20217(double) #0 {
top:
  %.inv = fcmp olt double %0, 0.000000e+00
  %spec.select = select i1 %.inv, double 0.000000e+00, double %0
;  @ In[12]:5 within `pos_fixed'
  ret double %spec.select
}


In [23]:
# 另一個方式是看assembly instructions的長度
@code_native pos(2.5)

	.text
; ┌ @ In[1]:4 within `pos'
	pushq	%rbp
	movq	%rsp, %rbp
; │┌ @ float.jl:503 within `<' @ float.jl:458
	vxorps	%xmm0, %xmm0, %xmm0
	vucomisd	%xmm1, %xmm0
; │└
	ja	L24
; │ @ In[1]:7 within `pos'
	vmovsd	%xmm1, (%rcx)
	movb	$1, %dl
	xorl	%eax, %eax
; │ @ In[1]:5 within `pos'
	popq	%rbp
	retq
L24:
	movb	$-126, %dl
	movl	$362709024, %eax        # imm = 0x159E8020
; │ @ In[1]:5 within `pos'
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)
; └


In [24]:
@code_native pos_fixed(2.5)

	.text
; ┌ @ In[12]:4 within `pos_fixed'
	pushq	%rbp
	movq	%rsp, %rbp
	vxorpd	%xmm1, %xmm1, %xmm1
	vmaxsd	%xmm0, %xmm1, %xmm0
; │ @ In[12]:5 within `pos_fixed'
	popq	%rbp
	retq
	nop
; └


In [25]:
# 型別穩定的另一項要素是, 函數中的變數不能隨著迴圈而改變型別, 此下的變數r就犯了這樣的錯
function sumsqrtn(n)
    r = 0
    for i = 1:n
        r = r + sqrt(i)
    end
    return r
end

sumsqrtn (generic function with 1 method)

In [26]:
# 查看@code_warntype, 發現有紅色警示
@code_warntype sumsqrtn(5)

Variables
  #self#::Core.Compiler.Const(sumsqrtn, false)
  n::Int64
  r::Union{Float64, Int64}
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Union{Float64, Int64}
1 ─       (r = 0)
│   %2  = (1:n)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = r::Union{Float64, Int64}
│   %11 = Main.sqrt(i)::Float64
│         (r = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return r


In [27]:
# 將初始的r改成Float64, 這樣型別就統一了
function sumsqrtn_fixed(n)
    r = 0.0
    for i = 1:n
        r = r + sqrt(i)
    end
    return r
end

sumsqrtn_fixed (generic function with 1 method)

In [28]:
@code_warntype sumsqrtn_fixed(5)

Variables
  #self#::Core.Compiler.Const(sumsqrtn_fixed, false)
  n::Int64
  r::Float64
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Float64
1 ─       (r = 0.0)
│   %2  = (1:n)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = r::Float64
│   %11 = Main.sqrt(i)::Float64
│         (r = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return r


In [36]:
function simdsum(x)
    s = 0
    @simd for v in x
        s += v
    end
    return s
end

simdsum (generic function with 1 method)

In [37]:
function simdsum_fixed(x)
    s = zero(eltype(x))
    @simd for v in x
        s += v
    end
    return s
end

simdsum_fixed (generic function with 1 method)

In [39]:
# 儘管Julia會將那些簡單的非型別穩定函數優化, 但在一些複雜的情況是沒辦法做到的
# 因此型別穩定對執行速度還是至關重要
using BenchmarkTools
a = rand(Float64, 10^6)
@btime simdsum(a)

  1.167 ms (1 allocation: 16 bytes)


499866.754881834

In [41]:
@btime simdsum_fixed(a)

  291.200 μs (1 allocation: 16 bytes)


499866.7548818324

In [43]:
# 假定這個函數可以輸入"Int64"or"Float64"
# 雖然輸入的型別都是字串, 但輸出的型別卻會隨著輸入的內容而不同
function string_zeros(s::AbstractString)
    n = 1000_000
    x = s=="Int64" ?
        Vector{Int64}(undef, n) :
        Vector{Float64}(undef, n)
    for i in 1:length(x)
        x[i] = 0
    end
    return x
end

string_zeros (generic function with 1 method)

In [45]:
# 發現過多的記憶體被指派, 原因是因為在迴圈部分, 我們沒辦法在編譯階段就知道x的型別
# 導致程式碼沒辦法被最佳化使用
@btime string_zeros("Int64");

  19.660 ms (999491 allocations: 22.88 MiB)


In [49]:
# 我們須要做的就是讓編譯器在迴圈操作的地方能知道x的型別
# 做法是將迴圈從函數中切分出來, 自成一個函數
function string_zeros_stable(s::AbstractString)
    n = 1000_000
    x = s=="Int64" ?
        Vector{Int64}(undef, n) :
        Vector{Float64}(undef, n)
    return fill_zeros(x)
end
function fill_zeros(x)
    for i in 1:length(x)
        x[i] = 0
    end
    return x
end

fill_zeros (generic function with 1 method)

In [50]:
@btime string_zeros_stable("Int64");

  1.323 ms (2 allocations: 7.63 MiB)


In [1]:
# 一般來說, 我們不需要為了運行速度而限制arguement或variable的型別
# 為了寫出通用一點的函數, 這些限制越少越好, 編譯器自己會判定所需的型別
# 但若是創建儲存空間, 如arrays, dictionaries, 則須要指定明確的型別

In [3]:
a = Int64[1,2,3,4,5,6,7,8,9,10]
b = Number[1,2,3,4,5,6,7,8,9,10]

10-element Array{Number,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [5]:
function arr_sumsqr(x::Array{T}) where T <: Number
    r = zero(T)
    for i = 1:length(x)
        r = r + x[i] ^ 2
    end
    return r
end

arr_sumsqr (generic function with 1 method)

In [11]:
@btime arr_sumsqr(a)

  22.066 ns (0 allocations: 0 bytes)


385

In [12]:
# 因為a儲存時使用了特定的型別, 編譯器知道該用多大的空間來儲存
# 但由於b是在抽象型別底下, 編譯器不知道該用多大的空間, 因此儲存了指向實際資料的指標
# 而實際資料則存在其他某個地方, 每次讀取都需要再經由指標指向實際資料
@btime arr_sumsqr(b)

  335.455 ns (0 allocations: 0 bytes)


385

In [13]:
# 同樣的道理也適用於composite type
struct Point
    x
    y
end

In [14]:
struct ConcretePoint
    x::Float64
    y::Float64
end

In [15]:
function sumsqr_points(a)
    s = 0.0
    for x in a
        s = s + x.x^2 + x.y^2
    end
    return s
end

sumsqr_points (generic function with 1 method)

In [16]:
p_array = [Point(rand(), rand()) for i in 1:1000_000];
cp_array = [ConcretePoint(rand(), rand()) for i in 1:1000_000];

In [17]:
@btime sumsqr_points(p_array)

  53.784 ms (3000000 allocations: 45.78 MiB)


665840.1174959864

In [18]:
@btime sumsqr_points(cp_array)

  2.312 ms (1 allocation: 16 bytes)


666301.7839225329

In [19]:
# 上面使用特定型別的例子雖然在效能上有好的表現, 但卻缺發彈性
# 若我們想使用Float32或Float16的型別, 那上述的composite type就無法使用
# 因此我們可以改寫一下, 讓它適用所有Float型別, 而效能也能夠保持

In [20]:
# 一個可能會想到, 卻有問題的解法是
struct PointWithAbstrct
    x::AbstractFloat
    y::AbstractFloat
end
# 但這做法跟前面的Point型別有一樣的問題, 編譯器依舊無法最有效率地存取

In [21]:
struct ParametricPoint{T <: AbstractFloat}
    x::T
    y::T
end

In [23]:
pp_array = [ParametricPoint(rand(), rand()) for i in 1:1000_000];

In [24]:
@btime sumsqr_points(pp_array)

  2.313 ms (1 allocation: 16 bytes)


666894.2041506754